# Lab 14: Monitoring & Health Checks**Duration:** 45 minutes  **Objective:** Implement comprehensive monitoring and health checks## 🎯 Learning Objectives- Implement health check endpoints- Monitor key metrics- Set up alerting- Create dashboards- Implement SLA monitoring

In [ ]:
import redisimport timeimport jsonimport statisticsfrom datetime import datetime, timedeltafrom colorama import init, Fore, Styleinit(autoreset=True)r = redis.Redis(host='localhost', port=6379, decode_responses=True)print(f"{Fore.GREEN}✅ Lab 14: Monitoring & Health Checks{Style.RESET_ALL}")

## Health Check and Monitoring Implementation

In [ ]:
class HealthMonitor:    """Comprehensive health monitoring system."""        def __init__(self, redis_client):        self.r = redis_client        self.metrics = []        def check_health(self):        """Perform comprehensive health check."""        health_status = {            'timestamp': datetime.now().isoformat(),            'checks': {}        }                # Connectivity check        try:            self.r.ping()            health_status['checks']['connectivity'] = {'status': 'healthy'}        except:            health_status['checks']['connectivity'] = {'status': 'unhealthy'}                # Memory check        info = self.r.info('memory')        used = info['used_memory']        max_mem = info.get('maxmemory', 0)                if max_mem > 0:            usage_percent = (used / max_mem) * 100            if usage_percent > 90:                health_status['checks']['memory'] = {                    'status': 'critical',                    'usage': f"{usage_percent:.1f}%"                }            elif usage_percent > 75:                health_status['checks']['memory'] = {                    'status': 'warning',                    'usage': f"{usage_percent:.1f}%"                }            else:                health_status['checks']['memory'] = {                    'status': 'healthy',                    'usage': f"{usage_percent:.1f}%"                }        else:            health_status['checks']['memory'] = {                'status': 'healthy',                'used': info['used_memory_human']            }                # Performance check        start = time.perf_counter()        self.r.set('health:test', 'test')        self.r.get('health:test')        self.r.delete('health:test')        latency = (time.perf_counter() - start) * 1000                if latency > 10:            health_status['checks']['performance'] = {                'status': 'warning',                'latency_ms': latency            }        else:            health_status['checks']['performance'] = {                'status': 'healthy',                'latency_ms': latency            }                # Overall status        statuses = [check['status'] for check in health_status['checks'].values()]        if 'unhealthy' in statuses or 'critical' in statuses:            health_status['overall'] = 'unhealthy'        elif 'warning' in statuses:            health_status['overall'] = 'degraded'        else:            health_status['overall'] = 'healthy'                return health_status        def collect_metrics(self):        """Collect performance metrics."""        info = self.r.info()                metrics = {            'timestamp': datetime.now().isoformat(),            'connections': info['connected_clients'],            'ops_per_sec': info.get('instantaneous_ops_per_sec', 0),            'memory_used': info['used_memory'],            'memory_rss': info['used_memory_rss'],            'keys': self.r.dbsize()        }                self.metrics.append(metrics)                # Keep only last 100 metrics        if len(self.metrics) > 100:            self.metrics = self.metrics[-100:]                return metrics        def get_metrics_summary(self):        """Get summary of collected metrics."""        if not self.metrics:            return {}                ops_values = [m['ops_per_sec'] for m in self.metrics]                return {            'avg_ops_per_sec': statistics.mean(ops_values),            'max_ops_per_sec': max(ops_values),            'min_ops_per_sec': min(ops_values),            'current_connections': self.metrics[-1]['connections'],            'current_keys': self.metrics[-1]['keys']        }monitor = HealthMonitor(r)# Perform health checkprint(f"{Fore.CYAN}Health Check:{Style.RESET_ALL}")health = monitor.check_health()print(json.dumps(health, indent=2))# Collect metricsprint(f"\n{Fore.CYAN}Collecting Metrics:{Style.RESET_ALL}")for i in range(5):    metrics = monitor.collect_metrics()    print(f"  Sample {i+1}: {metrics['ops_per_sec']} ops/sec, {metrics['connections']} connections")    time.sleep(0.5)# Get summaryprint(f"\n{Fore.CYAN}Metrics Summary:{Style.RESET_ALL}")summary = monitor.get_metrics_summary()for key, value in summary.items():    print(f"  {key}: {value}")print("\n✅ Monitoring and health checks completed")